In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

testing_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████████████████████| 26421880/26421880 [00:21<00:00, 1249663.41it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|█████████████████████████████████| 29515/29515 [00:00<00:00, 142455.40it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|████████████████████████████| 4422102/4422102 [00:01<00:00, 2755796.88it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|█████████████████████████████████| 5148/5148 [00:00<00:00, 19125134.63it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(testing_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.int64


In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
        
)
print(f"Using {device} device")

# Defining the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [14]:
def train(dataloader: DataLoader, model: NeuralNetwork, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Back prop
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}, [{current:>5d}/{size}]")

In [10]:
def test(dataloader: DataLoader, model: NeuralNetwork, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [15]:
epochs = 5

for t in range(epochs):
    print(f'Epoch {t+1}\n------------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
------------------------------------
loss: 1.515778, [   64/60000]
loss: 1.483541, [ 6464/60000]
loss: 1.337033, [12864/60000]
loss: 1.412941, [19264/60000]
loss: 1.286928, [25664/60000]
loss: 1.297141, [32064/60000]
loss: 1.314124, [38464/60000]
loss: 1.237343, [44864/60000]
loss: 1.277050, [51264/60000]
loss: 1.180979, [57664/60000]
Test Error: 
 Accuracy: 64.1%, Avg loss: 1.203672 

Epoch 2
------------------------------------
loss: 1.280164, [   64/60000]
loss: 1.264408, [ 6464/60000]
loss: 1.103239, [12864/60000]
loss: 1.213137, [19264/60000]
loss: 1.079707, [25664/60000]
loss: 1.112992, [32064/60000]
loss: 1.141828, [38464/60000]
loss: 1.073024, [44864/60000]
loss: 1.118608, [51264/60000]
loss: 1.037908, [57664/60000]
Test Error: 
 Accuracy: 65.0%, Avg loss: 1.053992 

Epoch 3
------------------------------------
loss: 1.121885, [   64/60000]
loss: 1.126541, [ 6464/60000]
loss: 0.948434, [12864/60000]
loss: 1.087362, [19264/60000]
loss: 0.952360, [25664/60000]
loss: 0.988

In [18]:
torch.save(model.state_dict(), 'model.pth')
print('Save PyTorch model state to file model.pth')

Save PyTorch model state to file model.pth


In [19]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [21]:
classes = [
    'T-shirt/top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sadal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankel boot'
]

model.eval()
x, y = testing_data[0][0], testing_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankel boot", Actual: "Ankel boot"
